# Import modules

In [27]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

## MINIST Test (Using FastMINIST)

MixedNational Institute of Standards and Technology Database

In [28]:
# Check if cuda is enable
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [29]:
# Download training data and test data
# url = ""
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
# url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz"
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [30]:
# Model definitoin

class NetworkModel(nn.Module):
    def __init__(self):
        super(NetworkModel, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [31]:
model = NetworkModel().to(device)
print(model)

NetworkModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [32]:
# loss function and optimizer

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [33]:
# dataloaders

batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
for (X, y) in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [34]:
# training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # predition error
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # save every 100
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [35]:
# test function
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy： {(100 * correct):>0.1f}%, Average loss: {test_loss:>8f} \n")

In [36]:
epochs = 5
for i in range(epochs):
    print(f"---- Epoch {i + 1} ----")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done")

---- Epoch 1 ----
loss: 2.299933 [    0/60000]
loss: 2.299534 [ 6400/60000]
loss: 2.297386 [12800/60000]
loss: 2.300331 [19200/60000]
loss: 2.299818 [25600/60000]
loss: 2.296238 [32000/60000]
loss: 2.296278 [38400/60000]
loss: 2.291545 [44800/60000]
loss: 2.280433 [51200/60000]
loss: 2.296719 [57600/60000]
Test Error: 
 Accuracy： 25.8%, Average loss: 0.035893 

---- Epoch 2 ----
loss: 2.267848 [    0/60000]
loss: 2.273011 [ 6400/60000]
loss: 2.257908 [12800/60000]
loss: 2.280724 [19200/60000]
loss: 2.285402 [25600/60000]
loss: 2.262924 [32000/60000]
loss: 2.287923 [38400/60000]
loss: 2.269847 [44800/60000]
loss: 2.237982 [51200/60000]
loss: 2.262876 [57600/60000]
Test Error: 
 Accuracy： 29.6%, Average loss: 0.035390 

---- Epoch 3 ----
loss: 2.226375 [    0/60000]
loss: 2.239653 [ 6400/60000]
loss: 2.204443 [12800/60000]
loss: 2.241678 [19200/60000]
loss: 2.250006 [25600/60000]
loss: 2.206567 [32000/60000]
loss: 2.264889 [38400/60000]
loss: 2.220086 [44800/60000]
loss: 2.181340 [51200/

In [37]:
# saving model

torch.save(model.state_dict(), "dist/model.pth")
print("Saved PyTorch Model State to dist/model.pth")

Saved PyTorch Model State to dist/model.pth


In [ ]:
# load model